## Работа с текстами

#### Задачи бинарной классификации
    * Сентимент анализ (позитивный \ негативный текст)
    * Модель "фильтр" (спам\не спам; текст по "теме"\не по "теме")
    * Детекция парафраз


#### Задачи мультикласс классификации
    * Тематическая классификация документов (классификация новостей)
    * Распознавание сущностей

#### Задачи кластеризации
    * Поиск групп схожих текстов
    * Тематическое моделирование

#### Задача регрессии
    * Предсказание оценки отзыва (1-5 звёзд)

#### Задача генерации текста \ языкового моделирования
    * Машинный перевод
    * Генерация парафраз
    * Автозаполнение

## Как представлять тексты в формате X?

Датасет $\{ (x_i, y_i) \}_{i=1}^N $

$X$ -- ?

Каковы свойства X?

* Фиксированный размер строчек в матрице
* Столбцы представляют один и тот же признак

Основная идея: __словарь__ и его размерность.

Один элемент словаря - один уникальный токен


"a quick brown fox jumped over the lazy dog. the cat sat on a mat."

```python

vocabulary = {
        "a": 0,
        "the": 1,
        "quick": 2
        "brown": 3
        .
        .
}
```

### Как разбивать тексты на токены?

text = `"Blazingly fast tokenization"`

---

* По правилам: razdel.tokenize, nltk.tokenize

`[Blazingly, fast, tokenization]`


----
* Статистически: byte pair encoding

`['▁Bl', 'az', 'ingly', '▁fast', '▁token', 'ization', '!']`

### Основная цель  bpe - сокращение размера словаря без необходимости предобработки

* Всего слов в русском языке: ~145k
* Часто используемых слов: ~20k

## Нормализация слов - ещё один способ сокращения размера словаря

text = `"прыгать прыгал, да не упрыгал"`

прыгать --> прыгать
прыгал  --> прыгать
упрыгал --> упрыгать (или тоже прыгать)
 
```python

vocabulary = {
    "прыгать": 0,
    "да": 1,
    "не": 2
}
```

### Преобразование словаря в признаки

```python
corpus = [
    "the cat sat on the mat",
    "the cat is on the window",
    "we like the cat"
]

word2idx = {
    "the": 0,
    "cat": 1,
    "sat": 2,
    "on": 3,
    "mat": 4,
    "is": 5,
    "window": 6,
    "we": 7,
    "like": 8
}
```


```"the cat sat on the mat" ```

```[2 1 1 1 1 0 0 0 0]```


```
word2idx = {
    "the": 0,
    "cat": 1,
    "sat": 2,
    "on": 3,
    "mat": 4,
    "is": 5,
    "window": 6,
    "we": 7,
    "like": 8
}
```

```"we like the cat"``` --> ?



```мама мыла раму``` --> ```рама мыла маму```

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
     'This is the first document.',
     'This document is the second document.',
     'And this is the third one.',
     'Is this the first document?',
 ]

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()
array(['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third',
       'this'], )

print(X.toarray())
[[0 1 1 1 0 0 1 0 1]
 [0 2 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]]

In [11]:
vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2, 2))
X2 = vectorizer2.fit_transform(corpus)
vectorizer2.get_feature_names_out()

array(['and this', 'document is', 'first document', 'is the', 'is this',
       'second document', 'the first', 'the second', 'the third', 'third one',
       'this document', 'this is', 'this the'], ...)

print(X2.toarray())
[[0 0 1 1 0 0 1 0 0 0 0 1 0]
[0 1 0 1 0 1 0 1 0 0 1 0 0]
[1 0 0 1 0 0 0 0 1 1 0 1 0]
[0 0 1 0 1 0 1 0 0 0 0 0 1]]

# ТЕстовая выборка

Ещё одна проблема с мешком слов, что подход мешка слов не учитывает шум.
Некоторые токены частотнее других (гораздо).

Однако при этом мы не хотим терять информацию о частотных, но специфичных для некоторого текста токенах


TF-IDF

### Term Frequency (TF)

Сколько раз слово встретилось в документе, делённое на общее число слов в документе

$$tf_{ij} = \frac{n_{ij}}{\sum_{k}n_{ij}}$$

### Inverse Document Frequency (IDF)

Число текстов делённое на количество текстов, которые содержат число w.

Определяет вес редких слов среди документов в корпусе

$$ idf(w) = log(\frac{N}{df_{t}})$$

### TF-IDF

Вес некоторого слова пропорционален частоте употребления этого слова в документе и обратно пропорционален частоте употребления слова во всех документах коллекции. 

$$ w_{ij} = tf_{ij} * log(\frac{N}{df_{t}}) $$

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
     'This is the first document.',
     'This document is the second document.',
     'And this is the third one.',
     'Is this the first document?',
 ]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()
array(['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third',
       'this'], )
print(X.shape)
(4, 9)

In [ ]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([

    #    ... предобработка

    ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('clf', LogisticRegression()),
 ])